In [7]:
import pandas, urllib, datetime

import pandas.io.sql as psql
from sqlalchemy import create_engine

params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER=%s,%s;DATABASE=%s;UID=%s;PWD=%s'%('localhost', '1433', 'ThuctapSNP', 'sa', 'Matkhau1234'))
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s'%params)

In [8]:
item = pandas.read_sql('select * from ITEM', engine)
item_reefer = pandas.read_sql('select * from ITEM_REEFER', engine)
item_dangerous = pandas.read_sql('select * from ITEM_DANGEROUS', engine)
item_oog = pandas.read_sql('select * from ITEM_OOG', engine)
item_location = pandas.read_sql('select * from ITEM_LOCATION', engine)
yard_area = pandas.read_sql('select * from YARD_AREA', engine)
pregate_transact = pandas.read_sql('select * from PREGATE_TRANSACT', engine)

In [9]:
item = item.loc[
    (item['SITE_ID'].str.strip()=='CTL') &
    (item['ARR_BY'].str.strip()=='T') &
    (item['HIST_FLG'].str.strip()=='Y') &
    (item['FEL'].str.strip()=='F')
]
item = item.drop(['ITEM_TYPE'], axis=1)
item = item[['ITEM_KEY', 'ITEM_NO', 'LENGTH', 'ARR_TS', 'DEP_TS']]
item = item.reset_index(drop=True)

item_location = item_location[['ITEM_KEY', 'STACK', 'EXEC_TS']]


item_reefer['ITEM_TYPE'] = 'R'
item_reefer = pandas.merge(item_reefer, item, how='left', on='ITEM_KEY')
item_reefer = item_reefer[['ITEM_KEY', 'ITEM_TYPE']]

item_dangerous['ITEM_TYPE'] = 'D'
item_dangerous = pandas.merge(item_dangerous, item, how='left', on='ITEM_KEY')
item_dangerous = item_dangerous[['ITEM_KEY', 'ITEM_TYPE']]

item_oog['ITEM_TYPE'] = 'O'
item_oog = pandas.merge(item_oog, item, how='left', on='ITEM_KEY')
item_oog = item_oog[['ITEM_KEY', 'ITEM_TYPE']]

item_special = pandas.concat([item_reefer, item_dangerous, item_oog], axis=0)
item = pandas.merge(item, item_special, how='left', on='ITEM_KEY').fillna({'ITEM_TYPE' : 'G'})

In [135]:
item = pandas.merge(item, pregate_transact, how='inner', on='ITEM_KEY', suffixes=('_item', '_pregate'))

In [192]:
item['DURATION'] = item['DEP_TS'] - item['ARR_TS']
item['DURATION'] = item['DURATION'].apply(pandas.Timedelta.total_seconds)

In [159]:
item['STACK'] = None
item['EXEC_TS'] = None
for i in range((len(item))):
    row = item.loc[i]
    r_d = row['R_D']
    lstSTACK = []
    if r_d==None:
        continue
    elif r_d.strip() == 'R':
        year = row['ARR_TS'].to_pydatetime().year
        if year > 1900:
            item.loc[i, 'EXEC_TS'] = row['ARR_TS']
            lstSTACK = item_location.loc[
                (item_location['ITEM_KEY'] == row['ITEM_KEY']) &
                (item_location['EXEC_TS'] > row['ARR_TS'])
            ].sort_values(['EXEC_TS'], ascending=[True])['STACK'].tolist()
        else:
            item = item.drop(i)
            item = item.reset_index(drop=True)
    elif r_d.strip() == 'D':
        year = row['DEP_TS'].to_pydatetime().year
        if year > 1900:
            item.loc[i, 'EXEC_TS'] = row['DEP_TS']
            lstSTACK = item_location.loc[
                (item_location['ITEM_KEY'] == row['ITEM_KEY'])
            ].sort_values(['EXEC_TS'], ascending=[False])['STACK'].tolist()
        else:
            item = item.drop(i)
            item = item.reset_index(drop=True)
    while len(lstSTACK)>0 and lstSTACK[0]!=None:
        item.loc[i, 'STACK'] = lstSTACK[0]
        break
    else:
        lstSTACK.pop(0)

In [162]:
item.columns

Index(['ITEM_KEY', 'ITEM_NO_item', 'LENGTH', 'ARR_TS', 'DEP_TS', 'ITEM_TYPE',
       'ITEM_NO_pregate', 'CHARGE_BATCH_ID', 'EIR_ID', 'R_D',
       'OPERATION_METHOD', 'AREA', 'RECEIVAL_PLACE', 'ISO_CODE', 'CTR_LENGTH',
       'CTR_HEIGHT', 'CTR_TYPE', 'OPERATION_TYPE', 'LINE_OPER', 'TFC_CODE',
       'CONSIGNOR', 'PLACE_OF_DELIVERY', 'BOOK_NO', 'GROSS', 'VES_CD', 'FPOD',
       'VES_ID', 'OUT_VOYAGE', 'SEAL_NO', 'TEMPERATURE', 'FEL', 'CATEGORY',
       'MANAGE_BY', 'HIST_FLG', 'KEY_INTF', 'DURATION', 'STACK', 'EXEC_TS'],
      dtype='object')

In [165]:
item = item[['ITEM_KEY', 'ITEM_NO_item', 'LENGTH', 'AREA', 'STACK', 'R_D', 'DURATION', 'EXEC_TS', 'ITEM_TYPE']]

In [171]:
item = item.rename(columns={
    'ITEM_NO_item' : 'ITEM_NO'
})

In [172]:
item.columns

Index(['ITEM_KEY', 'ITEM_NO', 'LENGTH', 'AREA', 'STACK', 'R_D', 'DURATION',
       'EXEC_TS', 'ITEM_TYPE'],
      dtype='object')

In [168]:
import pandas.io.sql as psql

In [173]:
psql.to_sql(item, 'stg_TRANSACT', engine, if_exists='append', index=False)

-1

In [181]:
receipt = pregate_transact.loc[pregate_transact['R_D'].str.strip()=='R']
deport = pregate_transact.loc[pregate_transact['R_D'].str.strip()=='D']

In [197]:
sample = pandas.merge(item, receipt, how='inner', on='ITEM_KEY', suffixes=('_item', '_pregate'))
sample = pandas.merge(sample, item_location, how='inner', on='ITEM_KEY')

In [208]:
data = pandas.merge(item, deport, how='inner', on='ITEM_KEY', suffixes=('_item', '_pregate')) 
data = pandas.merge(data, item_location, how='inner', on='ITEM_KEY')

In [185]:
engine.dispose()

In [209]:
sample = sample[['ITEM_KEY', 'ITEM_NO_item', 'LENGTH', 'AREA', 'STACK', 'R_D', 'DURATION', 'EXEC_TS', 'ITEM_TYPE']]
data = data[['ITEM_KEY', 'ITEM_NO_item', 'LENGTH', 'AREA', 'STACK', 'R_D', 'DURATION', 'EXEC_TS', 'ITEM_TYPE']]

In [4]:
import pandas.io.sql as psql

In [2]:
params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER=%s,%s;DATABASE=%s;UID=%s;PWD=%s'%('localhost', '1433', 'DWH_ThuctapSNP', 'sa', 'Matkhau1234'))
dwh_engine = create_engine('mssql+pyodbc:///?odbc_connect=%s'%params)

In [3]:
d_location = pandas.read_sql('select * from d_LOCATION', stg_engine)

In [6]:
psql.to_sql(d_location, 'd_LOCATION', con=dwh_engine, index=False)

-1

In [10]:
stg_transact = pandas.read_sql('select * from stg_TRANSACT', engine)

In [23]:
sample = pandas.merge(stg_transact, d_location, how='inner', on=['AREA', 'STACK'])

In [17]:
def StripText(text):
    try:
        return text.strip()
    except:
        return text

In [21]:
d_location['AREA'] = d_location['AREA'].apply(StripText)

In [27]:
columns = list(sample.columns)
columns.remove('REGION')

In [29]:
sample = sample[columns]

In [33]:
lstGROUP = set([tuple(row) for i, row in sample[['EXEC_TS', 'LOC_ID', 'R_D', 'LENGTH', 'ITEM_TYPE']].iterrows()])

In [36]:
f_transact = pandas.DataFrame([list(i) for i in lstGROUP], columns=['EXEC_TS', 'LOC_ID', 'R_D', 'LENGTH', 'ITEM_TYPE'])

In [37]:
f_transact['QUANTITY'] = None
f_transact['SUM_DURATION'] = None

In [43]:
for i in range(len(f_transact)):
    row = f_transact.loc[i]
    filtered = sample.loc[
        (sample['EXEC_TS'] == row['EXEC_TS']) &
        (sample['LOC_ID'] == row['LOC_ID']) &
        (sample['LENGTH'] == row['LENGTH']) &
        (sample['ITEM_TYPE'] == row['ITEM_TYPE'])
    ]
    quantity = len(filtered)
    sum_duration = sum(filtered['DURATION'].tolist())
    f_transact.loc[i, 'QUANTITY'] = quantity
    f_transact.loc[i, 'SUM_DURATION'] = sum_duration

In [44]:
f_transact

,EXEC_TS,LOC_ID,R_D,LENGTH,ITEM_TYPE,QUANTITY,SUM_DURATION
0,2020-12-01 03:18:59,15,R,20.0,G,1,607597
1,2019-02-01 06:16:56,236,R,40.0,G,1,135584
2,2019-07-01 05:23:05,193,R,20.0,G,1,171422
3,2020-11-02 06:02:47,147,R,20.0,G,1,1597244
4,2020-09-01 01:23:46,335,R,40.0,R,1,165389
...,...,...,...,...,...,...,...
20797,2020-03-01 04:26:12,149,R,40.0,G,4,1867272
20798,2020-10-01 03:24:17,90,R,40.0,R,1,74008
20799,2019-10-04 21:19:35,15,R,20.0,G,1,5422134
20800,2020-04-01 15:23:29,158,R,40.0,G,1,107485


In [45]:
psql.to_sql(f_transact, 'f_TRANSACT', con=dwh_engine, index=False)

-1